In [5]:
import os
import sys
import pandas as pd
from sklearn.preprocessing import LabelEncoder

sys.path.append(os.path.abspath(os.path.join('../scripts')))
from config import Config
from file_handler import FileHandler
from eda import *

Config.FEATURES_PATH.mkdir(parents=True, exist_ok=True)

In [6]:
file_handler = FileHandler()

train_df = file_handler.read_csv(str(Config.DATASET_PATH / "eda/train.csv"))
test_df = file_handler.read_csv(str(Config.DATASET_PATH / "test.csv"))
store_df = file_handler.read_csv(str(Config.DATASET_PATH / "eda/store.csv"))

C:\Users\dell\OneDrive\Desktop\pharmaceutical-sales-prediction\scripts\file_handler.py:26: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, na_values=missing_values)


In [7]:
#Converting to string since it is a mixed data type
convert_to_string(train_df, ['StateHoliday'])

def merge(df, store):
    df_merge = pd.merge(df, store, on='Store')
    return df_merge

In [8]:
def get_part_of_month(day):
    if (day < 10):
        return 0
    elif(day < 20):
        return 1
    else:
        return 2

In [17]:
def extract_test_features(df):
    df = merge(df, store_df)
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].apply(lambda x: x.year)
    df['Month'] = df['Date'].apply(lambda x: x.month)
    df['DayOfMonth'] = df['Date'].apply(lambda x: x.day)
    df['WeekOfYear'] = df['Date'].apply(lambda x: x.weekofyear)
    df['weekday'] = df['DayOfWeek'].apply(lambda x: 0 if (x in [6, 7]) else 1)

    df = df[df['Open'] == 1]
    df["part_of_month"] = df["DayOfMonth"].apply(get_part_of_month)

    # since machines understand only numbers change categorical variables to numerical value
    lb = LabelEncoder()
    df['StateHoliday'] = lb.fit_transform(df['StateHoliday'])
    df['Assortment'] = lb.fit_transform(df['Assortment'])
    df['StoreType'] = lb.fit_transform(df['StoreType'])

    df = df.drop(columns=['Id', 'Date'], axis=1)
    return df

In [20]:
def extract_features(df):
    df = df[df['Open'] == 1]
    df["part_of_month"] = df["DayOfMonth"].apply(get_part_of_month)

    # since machines understand only numbers change categorical variables to numerical value
    lb = LabelEncoder()
    df['StateHoliday'] = lb.fit_transform(df['StateHoliday'])

    df = df.drop(columns=['Sales', 'Customers', 'Date'], axis=1)
    return df

def extract_sales(df):
    df = df[df['Open'] == 1]
    return df[["Sales"]]

def extract_customers(df):
    df = df[df['Open'] == 1]
    return df[["Customers"]]

In [21]:
train_features = extract_features(train_df)
test_features = extract_test_features(test_df)

target_sales = extract_sales(train_df)
target_customers = extract_customers(train_df)

train_features.to_csv(str(Config.FEATURES_PATH / "train_features.csv"), index=None)
test_features.to_csv(str(Config.FEATURES_PATH / "test_features.csv"), index=None)

target_sales.to_csv(str(Config.FEATURES_PATH / "target_sales.csv"), index=None)
target_customers.to_csv(str(Config.FEATURES_PATH / "target_customers.csv"), index=None)

C:\Users\dell\AppData\Local\Temp\ipykernel_7588\1483366059.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["part_of_month"] = df["DayOfMonth"].apply(get_part_of_month)
C:\Users\dell\AppData\Local\Temp\ipykernel_7588\1483366059.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['StateHoliday'] = lb.fit_transform(df['StateHoliday'])


In [12]:
train_df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,DayOfMonth,WeekOfYear,weekday
0,1,5,2015-07-31,5263,555,1,1,0,1,2015,7,31,31,1
1,2,5,2015-07-31,6064,625,1,1,0,1,2015,7,31,31,1
2,3,5,2015-07-31,8314,821,1,1,0,1,2015,7,31,31,1
3,4,5,2015-07-31,13995,1498,1,1,0,1,2015,7,31,31,1
4,5,5,2015-07-31,4822,559,1,1,0,1,2015,7,31,31,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1,2013,1,1,1,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1,2013,1,1,1,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1,2013,1,1,1,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1,2013,1,1,1,1
